## EPCIS Signing Example

### Import libary

Make sure you have installed the epcis-signing package by executing `npm i epcis-signing`
If you are using a commonJS environment make sure to have installed `esm-hook` as well.

In [1]:
require('esm-hook');

const epcisSigning = require('epcis-signing');
const Ed25519VerificationKey2020 = require('@digitalbazaar/ed25519-verification-key-2020');

[Function (anonymous)]

### Create a dummy EPICS Event

For this example we will use an ObjectEvent

In [2]:
const epcisEvent = {
    "@context": [
      "https://ref.gs1.org/standards/epcis/2.0.0/epcis-context.jsonld"
    ],
    "type": "EPCISDocument",
    "schemaVersion": "2.0",
    "creationDate": "2023-05-09T12:00:00.000+01:00",
    "epcisBody": {
      "eventList": [
        {
          "type": "ObjectEvent",
          "eventID": "ni:///sha-256;e569889a66b4bd11650849abb2e1e8d2699e0d129357304bebda5fed1a68b5b3?ver=CBV2.0",
          "eventTime": "2023-05-09T11:04:03.15Z",
          "eventTimeZoneOffset": "+01:00",
          "epcList": [
              "https://id.gs1.org/01/04012345999990/21/XYZ-1234"
          ],
          "action": "OBSERVE",
          "bizStep": "repairing",
          "disposition": "conformant",
          "readPoint": {
              "id": "https://id.gs1.org/414/4012345000115"
          }
        }
      ]
    }
  }

### Generate a key pair

In order to sign the event you will need a cryptographic key pair. We create an async function to do so.

In [3]:
async function getKeyPair() {

    // generate the keyPair from seed
    let keyPair = await Ed25519VerificationKey2020.Ed25519VerificationKey2020.generate();

    // name the keyPair in order to make it resolvable
    keyPair.id = 'did:key:' + keyPair.publicKeyMultibase + '#' + keyPair.publicKeyMultibase;
    keyPair.controller = 'did:key:' + keyPair.publicKeyMultibase;

    return keyPair;

}

### Create signing function

Create an ansyc signing function combining the signing and the key creation.,

In [4]:
async function signEvent() {

    const keyPair = await getKeyPair();
    
    console.log('--------------------------------Generated key pair-----------------------------------');
    
    console.log(keyPair);

    return await epcisSigning.sign(epcisEvent, keyPair, 'https://test.com/vc/12345')

}

### Sign the event

Call the async `signEvent` function, wait for the promise to resolve and log the result.

In [5]:
signEvent()
    .then((signedCred) => {
        console.log('------------------------Signed event as verifiable credential------------------------');
        console.log(signedCred);
    });

Promise { <pending> }

--------------------------------Generated key pair-----------------------------------
Ed25519VerificationKey2020 {
  id: 'did:key:z6MknqxKBCxia9jUQwfASZTsuQAeg6QPwbzq3wVGKyS8mRnu#z6MknqxKBCxia9jUQwfASZTsuQAeg6QPwbzq3wVGKyS8mRnu',
  controller: 'did:key:z6MknqxKBCxia9jUQwfASZTsuQAeg6QPwbzq3wVGKyS8mRnu',
  revoked: undefined,
  type: 'Ed25519VerificationKey2020',
  publicKeyMultibase: 'z6MknqxKBCxia9jUQwfASZTsuQAeg6QPwbzq3wVGKyS8mRnu',
  privateKeyMultibase: 'zrv4LFUwJJNRBphFMpuhhGSKLgeEse2kJTzYZvhkDmeqChcWk7z3xm9PrHNSufAnvgL7J4pXAjjLSdCgpxSsvoZuLz1'
}
------------------------Signed event as verifiable credential------------------------
{
  '@context': [
    'https://www.w3.org/2018/credentials/v1',
    'https://w3id.org/security/suites/ed25519-2020/v1'
  ],
  type: [ 'VerifiableCredential' ],
  id: 'https://test.com/vc/12345',
  issuer: 'did:key:z6MknqxKBCxia9jUQwfASZTsuQAeg6QPwbzq3wVGKyS8mRnu',
  issuanceDate: '2023-05-08T13:36:43Z',
  credentialSubject: {
    '@context': [
      'http